<a href="https://colab.research.google.com/github/thisishasan/prog_mod_b/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [319]:
!wget -nc 'https://raw.githubusercontent.com/thisishasan/prog_mod_b/refs/heads/main/data.csv'

File ‘data.csv’ already there; not retrieving.



In [320]:
import pandas as pd

df = pd.read_csv("data.csv")

df = df[['Principio Attivo', 'Descrizione Gruppo', 'Denominazione e Confezione', 'Titolare AIC', 'AIC', 'Codice Gruppo Equivalenza']]

# Display the first 5 rows of the DataFrame
print(df.head())

  Principio Attivo                     Descrizione Gruppo  \
0      Acamprosato  ACAMPROSATO 333MG 84 UNITA' USO ORALE   
1        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
2        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
3        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   
4        Acarbosio    ACARBOSIO 100MG 40 UNITA' USO ORALE   

            Denominazione e Confezione                    Titolare AIC  \
0  CAMPRAL*84 cpr riv gastrores 333 mg          BRUNO FARMACEUTICI SpA   
1               GLUCOBAY*40 cpr 100 mg                       BAYER SpA   
2              ACARPHAGE*40 cpr 100 mg          BRUNO FARMACEUTICI SpA   
3              ACARBOSIO*40 cpr 100 mg  TECNIMEDE SOC.TECNICO-MED.S.A.   
4              ACARBOSIO*40 cpr 100 mg                DOC GENERICI Srl   

        AIC Codice Gruppo Equivalenza  
0  34208013                       H0A  
1  26851016                       H1A  
2  38835144                       H1A  
3  39716182                 

In [321]:
df.isnull().sum()

,0
Principio Attivo,0
Descrizione Gruppo,0
Denominazione e Confezione,0
Titolare AIC,0
AIC,0
Codice Gruppo Equivalenza,0


In [322]:
df.duplicated().sum()

np.int64(0)

In [314]:
#!pip install pypdf
!pip install pymupdf

In [315]:
import requests
#from pypdf import PdfReader
import fitz  # PyMuPDF
import re

# Iterate over each row and get structured data from the API
for index, row in df.iterrows():

    aic = row['AIC']

    # Build the API URL
    url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query={aic}&spellingCorrection=true&page=0"

    # Make the GET request
    response = requests.get(url)

    # Check for successful response
    if response.status_code == 200:
        json_data = response.json()  # Parse the response as JSON

        # Access the nested data field
        content = json_data.get('data', {}).get('content', [])
        if len(content) == 0:
          continue

        # Print each item in the content list
        for item in content:
            id = item.get('id')
            codice_atc = item.get('codiceAtc')[0]
            df.at[index, 'ATC'] = codice_atc
            descrizione_atc = item.get('descrizioneAtc')
            codice_sis = item.get('medicinale').get('codiceSis')
            aic6 = item.get('medicinale').get('aic6')
            leaflet_url = f"https://medicinali.aifa.gov.it/it/#/it/organizzazione/{codice_sis}/farmaci/{aic6}/stampati/FI"
            pdf_url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/{codice_sis}/farmaci/{aic6}/stampati?ts=RCP"
            df.at[index, 'URL'] = leaflet_url
            df.at[index, 'PDF URL'] = pdf_url
            print(f"\n--- codiceAtc: {codice_atc} ---")
            print(f"\n--- descrizioneAtc: {descrizione_atc} ---")
            print(f"\n--- codiceSis: {codice_sis} ---")
            print(f"\n--- id: {id} ---")
            print(f"\n--- leafletUrl: {leaflet_url} ---")
            print(f"\n--- pdf_url: {pdf_url} ---")


            # Download the PDF
            response = requests.get(pdf_url)

            if response.status_code == 200:
                # Load and read the PDF
                pdf_path = 'rcp_downloaded.pdf'

                if response.status_code == 200:
                  with open("/content/rcp_downloaded.pdf", "wb") as f:
                      f.write(response.content)
                  print("PDF downloaded and saved as /content/rcp_downloaded.pdf")
                else:
                  print(f"Failed to download PDF. Status code: {response.status_code}")

                # creating a pdf reader object
                #doc = PdfReader(pdf_path)
                doc = fitz.open(pdf_path)

                # Combine all page text into one string
                full_text = ""
                #for page in doc.pages:
                for page in doc:
                    #full_text += page.extract_text().strip()
                    full_text += page.get_text()
                doc.close()

                # Normalize the text: fix words broken across lines
                text = re.sub(r'([a-zA-Z])\n([a-zA-Z])', r'\1 \2', full_text)  # fix broken words
                text = re.sub(r'\n+', '\n', text)
                text = re.sub(r'\s{2,}', ' ', text)  # collapse long spaces

                # Define section headings (number + title)
                sections = [
                    ("4.1", "Indicazioni terapeutiche"),
                    ("4.2", "Posologia e modo di somministrazione"),
                    ("4.3", "Controindicazioni"),
                    ("4.4", "Avvertenze speciali e precauzioni di impiego"),
                    ("4.5", "Interazioni con altri medicinali ed altre forme di interazione"),
                    ("4.6", "Fertilità, gravidanza e allattamento"),
                    ("4.7", "Effetti sulla capacità di guidare veicoli e sull’uso di macchinari"),
                    ("4.8", "Effetti indesiderati"),
                    ("4.9", "Sovradosaggio"),
                    ("6.2", "Incompatibilità")
                ]

                # Create a regex pattern for section headers
                section_regex = [
                    fr"({num}[\.\s]*{re.escape(title)})"
                    for num, title in sections
                ]
                pattern = '|'.join(section_regex)
                pattern = re.compile(pattern, flags=re.IGNORECASE)

                # Find all section starts
                matches = list(pattern.finditer(text))

                # Extract section contents
                extracted = {}
                for i in range(len(matches)):
                    start = matches[i].start()
                    end = matches[i+1].start() if i+1 < len(matches) else len(text)

                    header = matches[i].group().strip()
                    content = text[start:end].replace(header, '', 1).strip()

                    # Match section by header prefix (e.g. "4.2")
                    matched_section = next((f"{num} {title}" for num, title in sections if num in header), header)
                    extracted[matched_section] = content

                # Save each section in a separate variable
                for title, content in extracted.items():
                    section_name = title.replace(" ", "_").replace(".", "")  # Create a valid variable name
                    globals()[section_name] = content  # Dynamically assign the content to a variable
                    df.at[index, title] = content.replace(title," ").strip()

                    # Print the results
                    print(f"\n{'='*80}\n{title}\n{'='*80}\n{content}...\n")

    else:
        print(f"Request failed for AIC {aic}: Status Code {response.status_code}")

Streaming output truncated to the last 5000 lines.


--- codiceAtc: V03AH01 ---

--- descrizioneAtc: ['DIAZOSSIDO'] ---

--- codiceSis: 1117 ---

--- id: 0000010163 ---

--- leafletUrl: https://medicinali.aifa.gov.it/it/#/it/organizzazione/1117/farmaci/23060/stampati/FI ---

--- pdf_url: https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/1117/farmaci/23060/stampati?ts=RCP ---
PDF downloaded and saved as /content/rcp_downloaded.pdf

4.1 Indicazioni terapeutiche
Proglicem è indicato per il trattamento dell’ipoglicemia sintomatica da iperinsulinismo di diversa etiologia. Il medicinale si è dimostrato efficace nelle seguenti forme: - ipoglicemia idiopatica dell’infanzia: a) leucino-sensibile b) non classificata - tumori insulari funzionanti: a) maligni b) benigni se inasportabili - neoplasie extrapancreatiche determinanti ipoglicemia - ipoglicemia neonatale - tesaurismosi glicogenica - ipoglicemia ad etiologia sconosciuta Proglicem può essere usato in periodo preoperatorio come r

In [316]:
df = df.fillna("Not available")

In [317]:
df.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza,ATC,URL,PDF URL,4.1 Indicazioni terapeutiche,4.2 Posologia e modo di somministrazione,4.3 Controindicazioni,4.4 Avvertenze speciali e precauzioni di impiego,4.5 Interazioni con altri medicinali ed altre forme di interazione,"4.6 Fertilità, gravidanza e allattamento",4.7 Effetti sulla capacità di guidare veicoli e sull’uso di macchinari,4.8 Effetti indesiderati,4.9 Sovradosaggio,6.2 Incompatibilità
0,Acamprosato,ACAMPROSATO 333MG 84 UNITA' USO ORALE,CAMPRAL*84 cpr riv gastrores 333 mg,BRUNO FARMACEUTICI SpA,34208013,H0A,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
1,Acarbosio,ACARBOSIO 100MG 40 UNITA' USO ORALE,GLUCOBAY*40 cpr 100 mg,BAYER SpA,26851016,H1A,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
2,Acarbosio,ACARBOSIO 100MG 40 UNITA' USO ORALE,ACARPHAGE*40 cpr 100 mg,BRUNO FARMACEUTICI SpA,38835144,H1A,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
3,Acarbosio,ACARBOSIO 100MG 40 UNITA' USO ORALE,ACARBOSIO*40 cpr 100 mg,TECNIMEDE SOC.TECNICO-MED.S.A.,39716182,H1A,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
4,Acarbosio,ACARBOSIO 100MG 40 UNITA' USO ORALE,ACARBOSIO*40 cpr 100 mg,DOC GENERICI Srl,44155024,H1A,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available


In [318]:
df.to_csv('updated_data.csv', index=False)
print("\nUpdated CSV saved as updated_data.csv")


Updated CSV saved as updated_data.csv
